In [28]:
import pandas as pd
import geopandas as gp
import numpy as np
import rasterstats
import os
import fiona
import rasterio
import rasterio.mask

In [29]:
output_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files"
model_name = "sp"

In [30]:
# set building height
default_height = 20

In [31]:
# set project crs
crs = 2926

In [32]:
# set extent boundary
extent_path = f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/{model_name}_extent.shp"
extent = gp.read_file(extent_path)

In [33]:
# floodplain extent for clipping
floodplain_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/floodplain.shp"
floodplain = gp.read_file(floodplain_path)

In [47]:
extent[1:]

,id,geometry
1,3.0,"POLYGON ((1269069.169 201239.803, 1278116.746 ..."


In [48]:

floodplain_extent_intersect = gp.overlay(extent[1:], floodplain, how = "intersection")

In [50]:
# clip parcels to floodplain
parcel_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Building_Parcels.shp"
parcels = gp.read_file(parcel_path)
parcels.to_crs(crs, inplace = True)

parcels['centroid_geom'] = parcels.centroid
parcels.set_geometry('centroid_geom',inplace = True)
parcels_clip = gp.overlay(parcels, floodplain_extent_intersect, how = "intersection")

In [51]:
parcels_merge = pd.merge(parcels_clip, parcels[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
parcels_merge.set_geometry('geometry_y', inplace = True)

In [52]:
parcels_merge.drop(['geometry_x'], axis=1, inplace=True)
parcels_merge.to_file(os.path.join(output_path, f"{model_name}_neighborhood_parcels.shp"))

ROADS PROCESSING

In [53]:
# roads processing
roads_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_projected.shp"
roads = gp.read_file(roads_path)
roads_clip = gp.overlay(roads, extent, how='intersection')
roads_clip.to_crs(crs, inplace = True)
roads_dissolve = roads_clip.dissolve()
roads_dissolve.to_file(os.path.join(output_path, f"{model_name}_neighborhood_streets.shp"))

BUILDINGS PROCESSING

In [54]:
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

In [55]:
buildings['centroid_geom'] = buildings.centroid
buildings.set_geometry('centroid_geom',inplace = True)
buildings_clip = gp.overlay(buildings, floodplain_extent_intersect, how = "intersection")

In [56]:
buildings_merge = pd.merge(buildings_clip, buildings[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
buildings_merge['footprint_geom'] = buildings_merge['geometry_y']
buildings_merge.set_geometry('footprint_geom', inplace = True)

In [57]:
# fill 0 and NA values to default_height
buildings_merge['BP99_APEX'].fillna(default_height, inplace = True)
buildings_merge['BP99_APEX'] = np.where(buildings_merge['BP99_APEX']==0,default_height, buildings_merge['BP99_APEX'])
buildings_merge['BP99_APEX'] = buildings_merge['BP99_APEX'].astype(int)

In [58]:
buildings_merge.to_crs(crs, inplace = True)
buildings_merge['AREA'] = buildings_merge.area
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','AREA','footprint_geom']].to_file(os.path.join(output_path,f"{model_name}_neighborhood_buildings.shp"))

/tmp/ipykernel_341/3199497357.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','AREA','footprint_geom']].to_file(os.path.join(output_path,f"{model_name}_neighborhood_buildings.shp"))


In [59]:
buildings_merge['bounding_geom'] = buildings_merge.envelope
parcel_building_join = gp.sjoin(buildings_merge, parcels_merge, how = 'left')

In [60]:
parcel_building_join['geom'] = np.where((parcel_building_join['LAND_USE_1']=='Single Family') | (parcel_building_join['LAND_USE_1']=='Multi-Family'), parcel_building_join['bounding_geom'], parcel_building_join['footprint_geom'])
parcel_building_join.set_geometry('geom', inplace = True)

In [61]:
building_single = parcel_building_join[parcel_building_join['LAND_USE_1']=='Single Family']
building_multi = parcel_building_join[parcel_building_join['LAND_USE_1']=='Multi-Family']
building_res = pd.concat([building_single, building_multi])
building_res = building_res.sort_values(by=['PIN','AREA']).drop_duplicates(subset=['PIN'], keep='last')


In [62]:
building_other = parcel_building_join[parcel_building_join['LAND_USE_1']!='Single Family']
building_other = building_other[building_other['LAND_USE_1']!='Multi-Family']

In [63]:
buildings_all = pd.concat([building_other,building_res])

In [64]:
if model_name == 'puget':
    buildings_all = buildings_all[buildings_all['AREA']>=2500]
elif model_name == 'sp':
    buildings_all = buildings_all[buildings_all['AREA']>=500]

In [65]:
buildings_all.set_crs(crs = crs, inplace = True)
buildings_all[['OBJECTID_left', 'BP99_TYPE', 'BP99_APEX','geom']].to_file(os.path.join(output_path, f"{model_name}_neighborhood_buildings_bb_trim.shp"))

/tmp/ipykernel_341/1308071339.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  buildings_all[['OBJECTID_left', 'BP99_TYPE', 'BP99_APEX','geom']].to_file(os.path.join(output_path, f"{model_name}_neighborhood_buildings_bb_trim.shp"))


WATERCOURSE PROCESSING

In [11]:
watercourses_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Urban_Watercourses/Urban_Watercourses.shp"
wc = gp.read_file(watercourses_path)
print(len(wc))
print(len(extent))
wc_clip = gp.overlay(wc, extent[:1], how='intersection')
print(len(wc_clip))
wc_clip.to_crs(crs, inplace = True)
wc_dissolve = wc_clip.dissolve()
wc_dissolve.to_file(os.path.join(output_path, f"{model_name}_neighborhood_wc.shp"))

1757
2
0


/tmp/ipykernel_229/336586033.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:2926

  wc_clip = gp.overlay(wc, extent[:1], how='intersection')
/home/nweiss/miniconda3/envs/gis/lib/python3.11/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


CONTOUR PROCESSING

In [ ]:
# clip catherine's topo
contours_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/DV_GIS Topo Data/06_GIS Data/02_Clipping Layers/01_Duwamish Valley/DV King Country Contours_clipped.shp"
contours = gp.read_file(contours_path)
contours.to_crs(crs, inplace = True)
contours_clip = gp.overlay(contours, extent, how = 'intersection')

In [ ]:
contours_clip_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_contours_clip.shp"
contours_clip = gp.read_file(contours_clip_path)

In [ ]:
# if elevation > 0, take every contour 10
contours_below_SL = contours_clip[contours_clip['ELEVATION']<=20]
contours_above_SL = contours_clip[contours_clip['IDX_20FT']==20]
contours_above_SL = contours_above_SL[contours_above_SL['ELEVATION']>20]
contours_trimmed = pd.concat([contours_below_SL,contours_above_SL])

In [ ]:
contours_trimmed.to_file(os.path.join(output_path, "neighborhood_contours_clip_test.shp"))

TOPO PROCESSING
- merging bathymetry data with topography dem

In [ ]:
# mask topo_rast by extent
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/topo.tif"

with fiona.open(extent_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(topo_rast) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [ ]:
topo_outpath = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_topo.asc"

# convert to ascii file
out_meta.update({"driver": "AAIGrid",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(topo_outpath, "w", **out_meta) as dest:
    dest.write(out_image)

EXTRA PROCESSING FOR ASSIGNING ABSOLUTE BUILDING HEIGHTS FROM TERRAIN

In [ ]:
## used topo to raster tool with 5ft contours and 5 ft pixel resolution
# there may be better ways to get DEM / elevation values as raster
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/min_bath_topo.tif"
zonal_stats = rasterstats.zonal_stats(buildings_merge, topo_rast, stats="count min mean max median std")
stats_df = pd.DataFrame.from_dict(zonal_stats).reset_index()
buildings_merge.reset_index(inplace = True)
merged_heights = pd.merge(buildings_merge, stats_df, how = 'left', left_on = 'index', right_on = 'index')
merged_heights['height_upd'] = np.where((merged_heights['BP99_APEX'].isna()==False) | (merged_heights['mean'].isna()==False), default_height, (merged_heights['BP99_APEX']-merged_heights['mean']))
merged_heights.set_geometry('geometry_y')
merged_heights[['OBJECTID','BP99_APEX','mean','min','max','median','std','height_upd','geometry_y']].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/building_heights_upd.shp")